https://huggingface.co/TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ/discussions/11

In [1]:
#!pip install sentencepiece
#!pip install tokenizers -U
#!pip install transformers[sentencepiece] -U
#!pip install accelerate -U
#!pip install optimum

#https://huggingface.co/TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ/discussions/5



In [2]:
import os

# Get the current working directory
cwd = os.getcwd()
cwd
os.chdir('/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/')
cwd = os.getcwd()
cwd

'/media/skynet3/8tb_a/rwd_github_private/ICBeLLM'

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" #auto gets confused for some reason and puts it on the 2080
import torch
torch.cuda.device_count()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.device_count()
torch.cuda.get_device_name(device=None)

'NVIDIA GeForce RTX 3090 Ti'

In [4]:
import torch
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM

# If you've already downloaded the model, reference its location here:
#quantized_model_dir = "/path/to/TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ"
# Or to download it from the hub and store it in the Hugging Face cache directory:

#quantized_model_dir = "./data_temp/Wizard-Vicuna-13B-Uncensored-GPTQ/" #TheBloke_WizardLM-Uncensored-Falcon-40B-3bit-GPTQ
#quantized_model_dir = "TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ"
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(quantized_model_dir, use_fast=False)


/home/skynet3/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#FileNotFoundError: Could not find model in TheBloke/Wizard-Vicuna-13B-Uncensored-GPTQ
#https://huggingface.co/TheBloke/wizard-vicuna-13B-GPTQ/discussions/7
#quantized_model_dir = "TheBloke/stable-vicuna-13B-GPTQ"
#model_basename = "wizard-vicuna-13B-GPTQ-4bit.compat.no-act-order" #YOU HAVE TO DO THIS

#model_basename = "Wizard-vicuna-30B-Uncensored-GPTQ-4bit.act-order"
# path to directory containing local model
#https://huggingface.co/TheBloke/starcoderplus-GPTQ/discussions/3
quantized_model_dir = "TheBloke/Wizard-Vicuna-30B-Uncensored-GPTQ" #"/workspace/models/TheBloke_Wizard-Vicuna-30B-Uncensored-GPTQ"
model_basename = "Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order" #.safetensors
use_triton = False 

model = AutoGPTQForCausalLM.from_quantized(quantized_model_dir, 
                                           device="cuda:0", 
                                           use_triton=False,
                                           use_safetensors=True , 
                                           model_basename=model_basename,
                                           quantize_config=None,
                                           #torch_dtype=torch.bfloat16, 
                                           trust_remote_code=True
                                           ) #





The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The safetensors archive passed at /home/skynet3/.cache/huggingface/hub/models--TheBloke--Wizard-Vicuna-30B-Uncensored-GPTQ/snapshots/7aeb8db6d49732afa76dc7ce60c5c040dbbcf3ea/Wizard-Vicuna-30B-Uncensored-GPTQ-4bit.act.order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
skip module injection for FusedLlamaMLPForQuantizedModel not support integrate without triton yet.


In [6]:
model.hf_device_map #yup, it offloaded the last layers to the CPU. That's why it's so slow.

{'': 0}

In [7]:
prompt = "Ask Something here"
prompt_model = f'''### Human: {prompt}
### Assistant:'''
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)

print("*** Output ***")
print(tokenizer.decode(output[0]))
print("**************")

*** Output ***
<s> ### Human: Ask Something here
### Assistant: Sure, I can help you with anything related to the topic of this book. What would you like to know?</s>
**************


In [8]:
# Ok it continues to be about 4 characters per token
# print(input_ids.shape) #So this prompt and a little bit of story is about 939 tokens, got another 1k to play with
#print(len(prompt))
#5046/1287


Tune the prompt

In [84]:
prompt = """Create a semicolon separated list of countries and coalitions of countries mentioned in final sentence of the paragraph. If necessary, first perform coreference resolution and map ambigious pronouns or rerences to named entities mentioned earlier in the paragraph. If an entity such as a leader or a military unit represents a country then include the country that entity represents instead of the entity. Explain your reasoning and then give a final answer.

### Paragraph:
The Communist regime, which had attained power in Russia on 7 November 1917, opted to withdraw from World War I through a separate peace with Germany--the Treaty of Brest-Litovsk on 3 March 1918.

### Final Sentence:
The Western Allies were anxious to maintain an eastern front against Germany and wished to prevent the Germans from seizing large stocks of arms in Vladivostok.

### Thought:
The entities mentioned in the last sentence are "Western Allies", "Germany", and "Vladivostok".
"Western Allies" refers to a coalition of countries who are not explicitly listed yet in the paragraph and so Western Allies should be included.
"Germany" refers to a country and so Germany should be included.
"Vladivostok" refers to a city in Russia and so Russia should be included.

### Final Answer:
Western Allies; Germany; Russia

### Paragraph:
Three of the Austro-Hungarian Empire's successor states--Czechoslovakia, Yugoslavia, and Hungary--experienced a two-phase crisis from 27 March until 15 November 1921. Karl IV, the last Hapsburg Emperor of Austria-Hungary, abdicated in November 1918 and was exiled to Switzerland; but he did not abdicate his right to the throne. Early in 1921 the Conference of Ambassadors, acting for the Supreme Council (Big Ten) at the Paris Peace Conference, issued a decree forbidding the restoration of the Hapsburg family to the vacant Hungarian throne. Moreover, several European governments issued statements indicating what recourse they might have if a Hapsburg reascended the throne in Budapest. On 27 March 1921 Karl unexpectedly returned to his homeland and called upon the Regent of Hungary, Admiral Horthy, to transfer his powers to the rightful ruler. This act triggered a crisis for Czechoslovakia and Yugoslavia.

### Final Sentence:
The latter responded two days later by declaring that it would regard any attempt to restore Karl as a casus belli.

### Thought:
The entities mentioned in the last sentence are "The latter", an implied actor who might "attempt to restore Karl", and "Karl".
"The latter' refers to Yugoslavia from the previous sentence which is a country and so Yugoslavia should be included.
"Any attempt to restore Karl" refers to Hungary which is a country and so Hungary should be included.

### Final Answer:
Yugoslavia; Hungary

### Paragraph:
Cuba and the United States were the adversaries in a crisis from 15 to 24 April 1961. A series of incursions into Caribbean and Central American states, in which Cuba was perceived to be involved, took place in 1959-60 In addition, the U.S. had become increasingly concerned about the strong ties that had developed between Cuba and the USSR. Relations between the U.S. and Cuba were strained; and the latter appealed to the Security Council at the end of 1960 charging the U.S. with plans to mount an invasion of Cuba using Cuban exiles trained in Guatemala. Diplomatic relations between the two countries were broken on 3 January 1961. Cuba and the United States were the adversaries in a crisis from 15 to 24 April 1961. A series of incursions into Caribbean and Central American states, in which Cuba was perceived to be involved, took place in 1959-60 In addition, the U.S. had become increasingly concerned about the strong ties that had developed between Cuba and the USSR. Relations between the U.S. and Cuba were strained; and the latter appealed to the Security Council at the end of 1960 charging the U.S. with plans to mount an invasion of Cuba using Cuban exiles trained in Guatemala. Diplomatic relations between the two countries were broken on 3 January 1961.

### Final Sentence:
In early April 1961 President Kennedy authorized an invasion of Cuba by the exiles, reasoning that, in the event of a failure, the remaining force would be able to establish guerrilla bases in the Cuban mountains.

### Thought: 
The entities mentioned in that last sentence are "President Kennedy", "Cuba", "the exiles", and "the Cuban mountains".
"President Kennedy" refers to President John F. Kennedy who is leader who represents the United States which is a country and so United States should be included.
"Cuba" is a country and so Cuba should be included.
"The exiles" refers to Cuban Exiles which is independent group and not a country or representing a country and so should be excluded.
"The Cuban mountains" refers to a geographic feature that is part of Cuba and so Cuba should be included.

### Final Answer:
United States; Cuba"

### Paragraph:
India experienced a crisis over Hyderabad from 21 August to 18 September 1948. Hyderabad was the second largest princely state in the subcontinent, after Jammu and Kashmir. It is located in the geographic center of India and, in 1948, 80 percent of its population was Hindu while most of the political elite were Muslim. The nizam of Hyderabad did not favor accession to distant Pakistan.

### Final Sentence:
Nor did predominantly Hindu India attract him.
"""
prompt_model = f'''### Human: {prompt}
### Assistant:'''
input_ids = tokenizer(prompt_model, return_tensors="pt").input_ids.cuda()
#print(input_ids.shape) #Ok if we pass too long a prompt it goes OOM. We have to catch that. #2048
assert(input_ids.shape[0]+250 < 2048 )

In [85]:
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=250) #uh oh, 1966 was too big too. Do both the max tokens and prompt have to fit in the context window?

answer = tokenizer.decode(output[0])

In [31]:
#print(answer)

In [ ]:
print(answer.replace(prompt,'').replace('<s>### Human: \n',''))

import pandas as pd
ICBeLLM_xy = pd.read_csv("/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/ICBeLLM_xy.tsv", sep="\t")

ICBeLLM_xy.columns
import os
file="/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/yhat_prompt_entity_yes_no.tsv"
if os.path.exists(file):
    temp=pd.read_csv(file,sep="\t")
else:
    temp=ICBeLLM_xy[['crisno', 'sentence_number_int_aligned', 'crisis_text','sentence_span_text']].copy()
    temp.loc[:,'yhat_prompt_entity_yes_no']=None
    temp.loc[:,'yhat_prompt_entity_list_explicit']=None
    
#65 fails
#Grab the last 2k characters of the story to avoid oom
for i in range(ICBeLLM_xy.shape[0]):
    if pd.isna(temp['yhat_prompt_entity_yes_no'][i]):
        #prompt = "Story: " + ICBeLLM_xy['story_so_far'].fillna('').values[i][-2000:] + "\n Task: " + ICBeLLM_xy['prompt_entity_yes_no'].values[i] + "\nSentence: " + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\n### Response:"
        prompt = ICBeLLM_xy['prompt_entity_yes_no'].values[i] + "\n\n" + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\nAnswer:"
        #print(prompt)
        tokens = tokenizer(prompt, return_tensors="pt").to("cuda:0").input_ids
        output = model.generate(input_ids=tokens, max_new_tokens=1, do_sample=True, temperature=0.8)
        result=tokenizer.decode(output[0]).replace(prompt,'')
        #print(result)
        temp.loc[i,'yhat_prompt_entity_yes_no'] = result
        #print()

    if pd.isna(temp['yhat_prompt_entity_list_explicit'][i]):
        #prompt = "Story: " + ICBeLLM_xy['story_so_far'].fillna('').values[i][-2000:] + "\n Task: " + ICBeLLM_xy['prompt_entity_list_explicit'].values[i] + "\nSentence: " + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\n### Response:"
        prompt = ICBeLLM_xy['prompt_entity_list_explicit'].values[i] + "\n\n" + ICBeLLM_xy['sentence_span_text'].values[i] + "\n\nCountries and Organizations:"
        #print(prompt)
        tokens = tokenizer(prompt, return_tensors="pt").to("cuda:0").input_ids
        output = model.generate(input_ids=tokens, max_new_tokens=20, do_sample=True, temperature=0.8)
        result=tokenizer.decode(output[0]).replace(prompt,'')
        #print(result)
        temp.loc[i,'yhat_prompt_entity_list_explicit'] = result
        
    temp.to_csv("/media/skynet3/8tb_a/rwd_github_private/ICBeLLM/data_out/yhat_prompt_entity_yes_no.tsv", sep="\t", index=False)

  ICBeLLM_xy['prompt_entity_list_explicit']
[prompt]
ICBeLLM_xy['story_so_far'].fillna('').values[i] 
pd.isna(temp['yhat_prompt_entity_yes_no'][1])


temp['yhat_prompt_entity_yes_no'][0] = "asdasdasd"
